In [1]:
#  Imports

from src.parsers import HMParser
from src.repairs import PymooHMRepair
from src.scenes import PymooHMProblem

from src.metrics import CommunityMetrics

# ignore the warnings (only demonstration purposes!)
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Data parsing

data = HMParser(file_path='data/EC_V4.xlsx', ec_id=1)
data.parse()

In [3]:
# Pymoo imports

from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize
from pymoo.termination.default import DefaultSingleObjectiveTermination

algorithm = DE(pop_size=30,
               sampling=LHS(),
               variant="DE/rand/1/bin",
               CR=0.3,
               dither="vector",
               jitter=False,
               repair=PymooHMRepair(data=data))


# Default termination:
termination = DefaultSingleObjectiveTermination(xtol=1e-8,
                                                cvtol=1e-6,
                                                ftol=1e-6,
                                                period=1000,
                                                n_max_gen=2e1,
                                                n_max_evals=5e9)

customProblem = PymooHMProblem(data=data)

temp_res = minimize(customProblem,
                    algorithm,
                    termination=termination,
                    save_history=False,
                    verbose=True)

30
n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |       30 |  3.104912E+05 |  2.904377E+03
30
     2 |       60 |  2.537017E+05 |  2.904377E+03
30
     3 |       90 |  2.169779E+05 |  2.904377E+03
30
     4 |      120 |  1.802673E+05 |  2.904377E+03
30
     5 |      150 |  1.701975E+05 |  2.904377E+03
30
     6 |      180 |  1.534547E+05 |  2.904377E+03
30
     7 |      210 |  1.300144E+05 |  2.547782E+03
30
     8 |      240 |  1.032669E+05 |  2.547782E+03
30
     9 |      270 |  8.985193E+04 |  2.547782E+03
30
    10 |      300 |  7.643880E+04 |  2.534870E+03
30
    11 |      330 |  6.637748E+04 |  2.534870E+03
30
    12 |      360 |  5.628706E+04 |  2.477291E+03
30
    13 |      390 |  3.625125E+04 |  2.477291E+03
30
    14 |      420 |  3.622289E+04 |  2.440779E+03
30
    15 |      450 |  3.617873E+04 |  2.410607E+03
30
    16 |      480 |  2.610928E+04 |  2.380575E+03
30
    17 |      510 |  1.939543E+04 |  2.354254E+03
30
    18 |      540 |  1.604874E+04 |  2.354254E

In [4]:
# Obj function value

temp_res.F

array([2172.75488886])

In [5]:
# Solution

decoded_resources = customProblem.decode(temp_res.X)

In [6]:
# Show the decoded resources

decoded_resources

{'genActPower': [genActPower_00,
  genActPower_01,
  genActPower_02,
  genActPower_03,
  genActPower_04,
  genActPower_05,
  genActPower_06],
 'genExcActPower': [genExcActPower_00,
  genExcActPower_01,
  genExcActPower_02,
  genExcActPower_03,
  genExcActPower_04,
  genExcActPower_05,
  genExcActPower_06],
 'pImp': pImp,
 'pExp': pExp,
 'loadRedActPower': [loadRedActPower_00,
  loadRedActPower_01,
  loadRedActPower_02,
  loadRedActPower_03,
  loadRedActPower_04,
  loadRedActPower_05],
 'loadCutActPower': [loadCutActPower_00,
  loadCutActPower_01,
  loadCutActPower_02,
  loadCutActPower_03,
  loadCutActPower_04,
  loadCutActPower_05],
 'loadENS': [loadENS_00,
  loadENS_01,
  loadENS_02,
  loadENS_03,
  loadENS_04,
  loadENS_05],
 'loadActPower': [loadActPower_00,
  loadActPower_01,
  loadActPower_02,
  loadActPower_03,
  loadActPower_04,
  loadActPower_05],
 'storDchActPower': [storDchActPower_00,
  storDchActPower_01,
  storDchActPower_02],
 'storChActPower': [storChActPower_00, storCh

In [7]:
# Metrics

metric = CommunityMetrics()
print(metric)

Community Metrics available: 
- self_consumption
- prosumer_consumption_community
- total_produced
- total_consumed
- import_export_balance



In [8]:
# Check the metrics
metric.cmd_self_consumption(production=decoded_resources['genActPower'],
                            consumption=decoded_resources['loadActPower'])

0.7511071799622477

In [10]:
# Do a list of metrics at once

metric.check_metrics_2inputs(components_a=decoded_resources['genActPower'],
                             components_b=decoded_resources['loadActPower'],
                             metric_list=['self_consumption',
                                          'prosumer_consumption_community'])

{'self_consumption': 0.7511071799622477,
 'prosumer_consumption_community': 1.3313679148297612}

In [11]:
# Try to get an error
# Total consumption is not a valid metric with two components

metric.check_metrics_2inputs(components_a=decoded_resources['genActPower'],
                             components_b=decoded_resources['loadActPower'],
                             metric_list=['self_consumption',
                                          'prosumer_consumption_community',
                                          'total_consumed'])

The requested metric is not available with provided inputs:  total_consumed


{'self_consumption': 0.7511071799622477,
 'prosumer_consumption_community': 1.3313679148297612,
 'total_consumed': None}

In [12]:
# Do note that the following metrics do not make sense to be called like this.
# Use at own discretion

metric.check_metrics_1input(decoded_resources['loadActPower'],
                            metric_list=['total_consumed',
                                         'total_produced'])

{'total_consumed': 342.40988559999994, 'total_produced': 342.40988559999994}